In [4]:
#include <iostream>
#include <string>
#include <math.h>       /* cosh, log */
using namespace std;

In [1]:
gROOT->ProcessLine(".! export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/camilo/HEPTools/Delphes/delphes6PY3/");
//gROOT->ProcessLine(".! source /home/camilo/HEPTools/Delphes/delphes6PY3/DelphesEnv.sh")

In [2]:
//#include "/home/camilo/HEPTools/Delphes/delphes6PY3/libDelphes"
//#include "/home/camilo/HEPTools/Delphes/delphes6PY3/external/"

In [3]:
/*Cargo la librería de Delphes*/
gSystem->AddDynamicPath("/home/camilo/HEPTools/Delphes/delphes6PY3/");
gSystem->AddIncludePath("/home/camilo/HEPTools/Delphes/delphes6PY3/");
gSystem->AddIncludePath("/home/camilo/HEPTools/Delphes/delphes6PY3/external/");
gSystem->AddIncludePath("/home/camilo/HEPTools/Delphes/delphes6PY3/external/ExRootAnalysis/");
//gSystem->AddDynamicPath("/home/camilo/HEPTools/Delphes/delphes6PY3/external/");
//gSystem->AddDynamicPath("/home/camilo/HEPTools/Delphes/delphes6PY3/external/ExRootAnalysis/");
//gSystem->Load("/home/camilo/HEPTools/Delphes/delphes6PY3/libDelphes.so");
gSystem->Load("libDelphes");

Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeReader.h
   requested to autoload type ExRootTreeReader
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeWriter.h
   requested to autoload type ExRootTreeWriter
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeBranch.h
   requested to autoload type ExRootTreeBranch
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootResult.h
   requested to autoload type ExRootResult
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootClassifier.h
   requested to autoload type ExRootClassifier
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootFilter.h
   requested to autoload type ExRootFil

In [ ]:
/*Instancio un objeto TChain*/
TChain chain("Delphes");

In [ ]:
/*Creo las variables necesarias para el for*/
std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_2j_mu-nu/Events/run_";
std::string nom2="/tag_1_delphes_events.root";
std::string num, out;

In [ ]:
/*Llenado del chain con todos los runs*/
for (Int_t i=1; i<11; i=i+1 ) {
   num = std::to_string(i);
   if( i < 10 ) {
       out= nom1+"0"+num+nom2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * c = out.c_str();
/*pego todos los datos*/
    chain.Add(c);
}

In [ ]:
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);

In [ ]:
/*Leo el numero de entradas*/
Long64_t numberOfEntries = treeReader->GetEntries()

In [ ]:
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuon = treeReader->UseBranch("Muon");
TClonesArray *branchMissingET = treeReader->UseBranch("MissingET");
TClonesArray *branchJet = treeReader->UseBranch("Jet");

In [ ]:
// Inicializo los contadores
Int_t cut1=0, cut2=0, cut3=0, cut4=0, cut5=0;

In [ ]:
for(Int_t entry = 0; entry < numberOfEntries; ++entry)
    {
        // Se carga un evento espesifico para ser analizado
        treeReader->ReadEntry(entry);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJet->GetEntries() > 1)
        {
            ++cut1; //Cuantos eventos pasan el corte 1
            
            Int_t count=0;
            //Se cuentan los jets tageados como b
            for (Int_t i=0; i<branchJet->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJet->At(i);
                if(jet->BTag>0)
                {
                    ++count;
                }
    
            }

            if( count>1 )
            {
                ++cut2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1, *muon2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuon->GetEntries() > 1)
                {
                    ++cut3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1 = (Muon *) branchMuon->At(0);
                    muon2 = (Muon *) branchMuon->At(1);
                    //Si la carga es diferente
                    int charge = muon1->Charge * muon2->Charge;
                    if( charge < 0 )
                    {
                        ++cut4; //Cuantos eventos pasan el corte 4
                
                        double MassMuon=(muon1->P4()+muon2->P4()).M();
                        if( !((MassMuon>80)&&(MassMuon<100)) )
                        {
                            ++cut5; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }

In [ ]:
cout<<"#Raw ="<<numberOfEntries<<endl;
cout<<"#Cut1="<<cut1<<endl;
cout<<"#Cut2="<<cut2<<endl;
cout<<"#Cut3="<<cut3<<endl;
cout<<"#Cut4="<<cut4<<endl;
cout<<"#Cut5="<<cut5<<endl<<endl;


float EFFN1=(float)cut1/(float)numberOfEntries;
float EFFN2=(float)cut2/(float)cut1;
float EFFN3=(float)cut3/(float)cut2;
float EFFN4=(float)cut4/(float)cut3;
float EFFN5=(float)cut5/(float)cut4;


float EFFA1=(float)cut1/(float)numberOfEntries;
float EFFA2=(float)cut2/(float)numberOfEntries;
float EFFA3=(float)cut3/(float)numberOfEntries;
float EFFA4=(float)cut4/(float)numberOfEntries;
float EFFA5=(float)cut5/(float)numberOfEntries;

cout<<"EFFA1="<<EFFA1<<endl;
cout<<"EFFA2="<<EFFA2<<endl;
cout<<"EFFA3="<<EFFA3<<endl;
cout<<"EFFA4="<<EFFA4<<endl;
cout<<"EFFA5="<<EFFA5<<endl<<endl;


cout<<"EFFN1="<<EFFN1<<endl;
cout<<"EFFN2="<<EFFN2<<endl;
cout<<"EFFN3="<<EFFN3<<endl;
cout<<"EFFN4="<<EFFN4<<endl;
cout<<"EFFN5="<<EFFN5<<endl;

In [ ]:
355148
341827
126190
55688
55687
48077